In [1]:
from entsoe import EntsoePandasClient
import pandas as pd
import matplotlib.pyplot as plt

client = EntsoePandasClient(api_key="b18dfce9-f1e3-4d07-822f-4abd1438e602")

In [2]:
start = pd.Timestamp('20220901', tz='Europe/Amsterdam')
end = pd.Timestamp('20230228', tz='Europe/Amsterdam')

non_fbmc_countries = ['CH', 'GB', 'ES', 'IT_NORD', 'DK_1', 'NO_2']

gen_types = [
    'Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 
    'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 
    'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other',
    'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore'
]

standardized_gen_types = [
    'biomass', 'brown_coal', 'coal_gas', 'natural_gas', 'hard_coal', 'oil', 'oil_shale', 'peat', 'geothermal', 
    'hydro_pumped_storage', 'hydro_run_of_river', 'hydro_reservoir', 'marine', 'nuclear', 'other', 'other_renewable',
    'solar', 'waste', 'wind_offshore', 'wind_onshore'
]

reduced_gen_types = [
    'biomass', 'brown_coal', 'coal_gas', 'natural_gas', 'hard_coal', 'oil', 'hydro', 
    'nuclear', 'waste', 'other', 'solar', 'wind_onshore', 'wind_offshore', 'other_renewable'
]

In [3]:
def standardize_generation_data(df_gen):
    for gen_type in gen_types:
        if gen_type not in df_gen:
            df_gen[gen_type] = 0.0

    for gen_type in gen_types:
        if (gen_type, 'Actual Consumption') in df_gen:
            df_gen = df_gen.drop(columns=[(gen_type, 'Actual Consumption')], axis=1)

    df_gen = df_gen[gen_types]
    df_gen.columns = standardized_gen_types

    df_gen = df_gen.tz_localize(None)

    df_gen = df_gen.resample("H").agg(["mean"])
    df_gen.columns = standardized_gen_types
    
    return df_gen

In [12]:
# DAY-AHEAD PRICES

df_da_prices = pd.DataFrame()
for country_code in [x for x in non_fbmc_countries if x != "GB"]:
    df_da = client.query_day_ahead_prices(country_code, start=start, end=end)
    print(country_code)
    
    df_da = df_da.to_frame()
    df_da = df_da.rename(columns={
        0: country_code, 
    })

    df_da = df_da.tz_localize(None)
    df_da = df_da.resample("H").agg(["mean"])
    df_da.columns = [country_code]
    
    if df_da_prices.empty:
        df_da_prices = df_da.copy()
    else:
        df_da_prices[country_code] = df_da[country_code]

df_da_prices.to_excel("./optimisation/data/day_ahead_prices_non_fbmc.xlsx")

CH
ES
IT_NORD
DK_1
NO_2


In [18]:
# DEMAND

df_demand = pd.DataFrame()
for country_code in [x for x in non_fbmc_countries if x != "GB"]: # NO GB
    df_da = client.query_load_forecast(country_code, start=start, end=end)
    print(country_code)
    
    df_da = df_da.rename(columns={
        'Forecasted Load': country_code, 
    })
    
    df_da = df_da.tz_localize(None)
    df_da = df_da.resample("H").agg(["mean"])
    df_da.columns = [country_code]
    
    if df_demand.empty:
        df_demand = df_da.copy()
    else:
        df_demand[country_code] = df_da[country_code]
        
df_demand.to_excel("./optimisation/data/demand_non_fbmc.xlsx")

CH
ES
IT_NORD
DK_1
NO_2


In [20]:
# GENERATION

with pd.ExcelWriter("./optimisation/data/generation_non_fbmc.xlsx") as writer:  
    for country_code in [x for x in non_fbmc_countries if x != "GB"]: # NO GB
        df_gen = client.query_generation(country_code, start=start,end=end)
        print(country_code)
        
        df_gen = standardize_generation_data(df_gen)
        df_gen.loc[:, 'hydro'] = df_gen['hydro_pumped_storage'] + df_gen['hydro_run_of_river'] + df_gen['hydro_reservoir']
        df_gen = df_gen[reduced_gen_types]
   
        df_gen.to_excel(writer, sheet_name=country_code) 

CH
ES
IT_NORD
DK_1
NO_2


In [22]:
# INSTALLED CAPACITIES

start_inst = pd.Timestamp('20200101', tz='Europe/Amsterdam')
end_inst = pd.Timestamp('20230101', tz='Europe/Amsterdam')

df_capacities = pd.DataFrame()
for country_code in [x for x in non_fbmc_countries if x != "IT_NORD"]: # NO IT_NORD
    df_cap = client.query_installed_generation_capacity(country_code, start=start_inst, end=end_inst)
    print(country_code)
    
    for gen_type in gen_types:
        if gen_type not in df_cap:
            df_cap[gen_type] = 0.0

    df_cap = df_cap[gen_types]
    df_cap.columns = standardized_gen_types
    
    df_cap = df_cap.iloc[-1].to_frame()
    df_cap = df_cap.rename(columns={
        df_cap.columns[0]: country_code
    })
    
    if df_capacities.empty:
        df_capacities = df_cap.copy()
    else:
        df_capacities[country_code] = df_cap[country_code]
         
df_capacities = df_capacities.drop([df_capacities.index[6], df_capacities.index[7], df_capacities.index[8], df_capacities.index[12]])
df_capacities = df_capacities.rename(index={'hydro_pumped_storage': 'hydro'})
    
for column in df_capacities:
    df_capacities.loc['hydro', column] = df_capacities.loc['hydro', column] + df_capacities.loc['hydro_run_of_river', column] + df_capacities.loc['hydro_reservoir', column]

df_capacities = df_capacities.drop([df_capacities.index[7], df_capacities.index[8]])

df_capacities = df_capacities.reindex(reduced_gen_types)

df_capacities.to_excel("./optimisation/data/installed_capacities_non_fbmc.xlsx")

CH
GB
ES
DK_1
NO_2


In [23]:
# GENERATION OUTAGES

with pd.ExcelWriter("./optimisation/data/generation_outages_non_fbmc.xlsx") as writer:
    for country_code in non_fbmc_countries:
        print(country_code)
        
        try:
            df_out_gen = client.query_unavailability_of_generation_units(country_code, start=start, end=end)
            df_out_gen = df_out_gen.tz_localize(None)
            df_out_gen['end'] = df_out_gen['end'].dt.tz_localize(None)
            df_out_gen['start'] = df_out_gen['start'].dt.tz_localize(None)

            df_outages_zone = pd.DataFrame()

            for gen_date in df_gen.index:
                df_out_gen_test = df_out_gen[(df_out_gen.start <= gen_date) & (df_out_gen.end >= gen_date) & (df_out_gen.docstatus != 'Cancelled')]

                for gen_type in standardized_gen_types:
                        df_outages_zone.loc[gen_date, gen_type] = 0

                for index, row in df_out_gen_test.iterrows():
                    df_outages_zone.loc[gen_date, standardized_gen_types[gen_types.index(row['plant_type'])]] += float(row['nominal_power']) - float(row['avail_qty'])
                    df_outages_zone.loc[gen_date, 'hydro'] = df_outages_zone.loc[gen_date, 'hydro_pumped_storage'] + df_outages_zone.loc[gen_date, 'hydro_run_of_river'] + df_outages_zone.loc[gen_date, 'hydro_reservoir']

            df_outages_zone = df_outages_zone[reduced_gen_types]
            df_outages_zone.to_excel(writer, sheet_name=country_code)
        except:
            print("An exception occurred")
            df_outages_zone = pd.DataFrame()
            for gen_date in df_gen.index:
                for gen_type in reduced_gen_types:
                    df_outages_zone.loc[gen_date, gen_type] = 0
            df_outages_zone.to_excel(writer, sheet_name=country_code)

CH
GB
ES
IT_NORD
DK_1
NO_2


In [26]:
# RENEWABLE GENERATION FORECAST

df_ren_gen = pd.DataFrame()

for country_code in [x for x in non_fbmc_countries if x != "GB"]: # NO GB
    df_gen = client.query_wind_and_solar_forecast(country_code, start=start,end=end)
    print(country_code)
    
    if 'Solar' not in df_gen:
        df_gen['Solar'] = 0
    if 'Wind Onshore' not in df_gen:
        df_gen['Wind Onshore'] = 0
    if 'Wind Offshore' not in df_gen:
        df_gen['Wind Offshore'] = 0

    df_gen = df_gen.tz_localize(None)
    df_gen = df_gen.resample("H").agg(["mean"])

    df_ren_gen[country_code] = df_gen['Solar'] + df_gen['Wind Onshore'] + df_gen['Wind Offshore']
    
df_ren_gen.to_excel("./optimisation/data/renewable_generation_non_fbmc.xlsx")

CH
ES
IT_NORD
DK_1
NO_2


In [9]:
# UK: DAY-AHEAD PRICES
df_da_prices = pd.read_excel("./optimisation/data/day_ahead_prices_non_fbmc.xlsx", index_col=0)

df_uk_settlements = pd.read_excel("./uk_settlements.xlsx", index_col=0)
df_uk_settlements = df_uk_settlements[["EPEX UK EUR"]]
df_uk_settlements = df_uk_settlements.tz_localize(None)
df_uk_settlements = df_uk_settlements.resample("H").agg(["mean"])
df_uk_settlements.columns = ['epex']

df_da_prices['GB'] = df_uk_settlements['epex']
df_da_prices = df_da_prices[non_fbmc_countries]

df_da_prices.to_excel("./optimisation/data/day_ahead_prices_non_fbmc.xlsx")

In [28]:
# UK: DEMAND

df_demand = pd.read_excel("./optimisation/data/demand_non_fbmc.xlsx", index_col=0)

df_uk_gridwatch = pd.read_csv("./uk_gridwatch.csv", index_col=1)
df_uk_gridwatch.index = pd.to_datetime(df_uk_gridwatch.index)
df_uk_gridwatch = df_uk_gridwatch[[
    ' demand', ' coal', ' nuclear', ' ccgt', ' pumped', ' hydro', ' biomass', ' oil', ' ocgt', ' wind', ' solar' 
]]
df_uk_gridwatch = df_uk_gridwatch.tz_localize(None)
df_uk_gridwatch = df_uk_gridwatch.resample("H").agg(["mean"])
df_uk_gridwatch.columns = [
    'demand', 'coal', 'nuclear', 'ccgt', 'pumped', 'hydro', 'biomass', 'oil', 'ocgt', 'wind', 'solar'  
]

df_demand['GB'] = df_uk_gridwatch['demand']
df_demand = df_demand[non_fbmc_countries]
df_demand.to_excel("./optimisation/data/demand_non_fbmc.xlsx")

In [33]:
# UK: GENERATION

df_gen = pd.read_excel("./optimisation/data/generation_non_fbmc.xlsx", sheet_name='CH', index_col=0)
df_gen_uk = pd.DataFrame()
df_gen_uk.index = df_gen.index

df_gen_uk['biomass'] = df_uk_gridwatch['biomass']
df_gen_uk['brown_coal'] = 0
df_gen_uk['coal_gas'] = 0
df_gen_uk['natural_gas'] = df_uk_gridwatch['ccgt'] + df_uk_gridwatch['ocgt']
df_gen_uk['hard_coal'] = df_uk_gridwatch['coal']
df_gen_uk['oil'] = df_uk_gridwatch['oil']
df_gen_uk['hydro'] = df_uk_gridwatch['pumped'] + df_uk_gridwatch['hydro']
df_gen_uk['nuclear'] = df_uk_gridwatch['nuclear']
df_gen_uk['waste'] = 0
df_gen_uk['other'] = 0

df_gen_uk['solar'] = df_uk_gridwatch['solar']
df_gen_uk['wind_onshore'] = 0
df_gen_uk['wind_offshore'] = df_uk_gridwatch['wind']
df_gen_uk['other_renewable'] = 0

df_gen_uk = df_gen_uk[reduced_gen_types]
df_gen_uk.to_excel("./optimisation/data/generation_uk_non_fbmc.xlsx")

In [39]:
# UK: RENEWABLE GENERATION (FORECAST)

df_ren_gen = pd.read_excel("./optimisation/data/renewable_generation_non_fbmc.xlsx", index_col=0)
df_ren_gen['GB'] = df_uk_gridwatch['wind'] + df_uk_gridwatch['solar']
df_ren_gen = df_ren_gen[non_fbmc_countries]
df_ren_gen.to_excel("./optimisation/data/renewable_generation_non_fbmc.xlsx")

In [44]:
# IT: INSTALLED CAPACITIES

df_capacities = pd.read_excel("./optimisation/data/installed_capacities_non_fbmc.xlsx", index_col=0)

df_capacities.loc['biomass', 'IT_NORD'] = 1534
df_capacities.loc['brown_coal', 'IT_NORD'] = 0
df_capacities.loc['coal_gas', 'IT_NORD'] = 2072
df_capacities.loc['natural_gas', 'IT_NORD'] = 44215
df_capacities.loc['hard_coal', 'IT_NORD'] = 5576
df_capacities.loc['oil', 'IT_NORD'] = 1543
df_capacities.loc['hydro', 'IT_NORD'] = 10553 + 4440 + 7256
df_capacities.loc['nuclear', 'IT_NORD'] = 0
df_capacities.loc['waste', 'IT_NORD'] = 120
df_capacities.loc['other', 'IT_NORD'] = 869 + 853

df_capacities.loc['solar', 'IT_NORD'] = 5431
df_capacities.loc['wind_onshore', 'IT_NORD'] = 11204
df_capacities.loc['wind_offshore', 'IT_NORD'] = 30
df_capacities.loc['other_renewable', 'IT_NORD'] = 0

df_capacities = df_capacities[non_fbmc_countries]
df_capacities.to_excel("./optimisation/data/installed_capacities_non_fbmc.xlsx")